**Работа с портом в docker**

Порт это "окно" которое позволяет программе запущенной на компьютере связыватся с интернетом.

# "Прокинуть" порт

При выполнении `docker run` нужно укзать опцию `-p` и затем передать `<порт на хосте>:<порт в контейнере>`.

### Базовый пример

Для примера рассмотрим clickhouse, сначала просто его запустим:

In [1]:
%%bash
docker run --rm -d --name db_port yandex/clickhouse-server
# проверка того отвечает ли порт 8123 (видимо базовый для clickhouse)
curl localhost:8123
docker stop db_port

9501fc87dee9210094e0b28076a4b194aad159c3490f76a7ed31498196ba8e68


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (7) Failed to connect to localhost port 8123 after 0 ms: Connection refused


db_port


В результате `curl` оветил `Failed ...`.

А теперь продемонстрируем тоже самое, но укажем опцию `-p 8123:8123`. Теперь то, спросив что-то с порта 8123 мы получили `OK`.

In [4]:
%%bash
docker run --rm -d --name db_port -p 8123:8123 yandex/clickhouse-server
# clickshouse нужно немнго времени для того, чтобы развернуться
sleep 10
curl localhost:8123
docker stop db_port

76e77328a72b90ca8b7ec8cddd79582b7498de70f6ec502a0cff1cd57e829568


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100     4    0     4    0     0   2711      0 --:--:-- --:--:-- --:--:--  4000


Ok.
db_port


### Прокидывание нескольких портов

Для того, чтобы прокинуть несколько портов в контенер опцию `-p` следует исопльзовать несколько раз. Так в следующем примере мы к одному и тому-же контейнеру подвязываем сразу два контейнера и, затем, спрашиваем их чере curl.

In [20]:
%%bash
docker run --rm -d --name db_port -p 8123:8123 -p 8124:8123 yandex/clickhouse-server
curl localhost:8123
curl localhost:8124
docker stop db_port

004bdedc8fce6f5945aef97ff822813f67d7edb7036218e182d13f7bf3e6cf72


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100     4    0     4    0     0   2762      0 --:--:-- --:--:-- --:--:--  4000


Ok.


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100     4    0     4    0     0   4040      0 --:--:-- --:--:-- --:--:--  4000


Ok.
db_port


# `EXPOSE`

Это инструкция dockerfile, которая подскзывает пользователю какой порт будет слушать программа завернутая в контейнер. Рассмотрим поле "ports" команды `ps` от docker. Так, там перечислены, порты которые могут быть использованы clickhouse.

In [9]:
%%bash
docker run --rm -d --name db_ports yandex/clickhouse-server
docker ps
docker stop db_ports

d26dfc13efc2fd37560d46b4af5e196412e48b2cc3e22e3f8de2ce708989a436
CONTAINER ID   IMAGE                      COMMAND            CREATED        STATUS                  PORTS                          NAMES
d26dfc13efc2   yandex/clickhouse-server   "/entrypoint.sh"   1 second ago   Up Less than a second   8123/tcp, 9000/tcp, 9009/tcp   db_ports
db_ports


А если среди слоев образа clickhouse искать ключевое слово `EXPOSE`, то получается, то получим:

In [7]:
%%bash
docker history yandex/clickhouse-server | grep EXPOSE

<missing>      13 months ago   /bin/sh -c #(nop)  EXPOSE 8123 9000 9009        0B        


Так, получается есть слой с инструкцией `EXPOSE` для которого пречислены все те-же числа. Ну, попробуем собрать простой dockerfile в котором используем эту конструкцию. Так, dockerfile принял вид:
```
FROM ubuntu
EXPOSE 1050
```

In [8]:
%%bash
docker run -itd --name expose_test --rm expose_test
docker ps
docker stop expose_test

6112abb33b2e5c863cef3b40e59aa0b19ae8523c8d8dc41c3f476b4bdfb05fd3
CONTAINER ID   IMAGE         COMMAND       CREATED        STATUS                  PORTS      NAMES
6112abb33b2e   expose_test   "/bin/bash"   1 second ago   Up Less than a second   1050/tcp   expose_test
expose_test


Так в порте видим заветные 1050.

# Детали

### Порт в контейнере 

В нутри самого контейнера порты будут открыты и без всяких опций. Так в примере ниже я поднимаю clickhouse и полсе дого спрашиваю `wget` (потому что `curl` по умолчанию не завезли).

In [15]:
%%bash
docker run -itd --name db_test --rm yandex/clickhouse-server
sleep 10
docker exec db_test wget localhost:8123
docker stop db_test

efee7438d3e4cae585b742cc1ab20d7051ce17004d1de0b88871dae31384496a


--2023-03-10 18:37:10--  http://localhost:8123/
Resolving localhost (localhost)... 127.0.0.1, ::1
Connecting to localhost (localhost)|127.0.0.1|:8123... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘index.html’

     0K                                                         611K=0s

2023-03-10 18:37:10 (611 KB/s) - ‘index.html’ saved [4]



db_test


### Контейнеры с одним портом

Думаю очевидно, что нельзя на один и тот-же порт повесить более двух контейнеров. Однако, есть способ обойти эту особенность, нужно к опции `-p` для порта хоста указать другой порт, в то время как в контейнере использовать все тот-же.

Так для начала, пример с ошибкой - пытаеися повесить два контейнера на один и тот-же порт.

In [17]:
%%bash
docker run -itd --name db_test1 --rm -p 8123:8123 yandex/clickhouse-server
docker run -itd --name db_test2 --rm -p 8123:8123 yandex/clickhouse-server 
sleep 10
docker stop db_test1

6b56c296cfcc1aa9f46968767f9502de695f474df65d60aad2716eef8813daee
07fdf42a22e2045f74785e3247a51b8c75883dc701da8427055a66190953a356


docker: Error response from daemon: driver failed programming external connectivity on endpoint db_test2 (edaa15bb3cba3dc8dec4558bdc32ea5a9a9978560b7cc5678dd53216c6ba187b): Bind for 0.0.0.0:8123 failed: port is already allocated.


db_test1


Путь исправления, соотвественно. Второй контейнер при запуске использует порт `8124` контейнера.

In [18]:
%%bash
docker run -itd --name db_test1 --rm -p 8123:8123 yandex/clickhouse-server
docker run -itd --name db_test2 --rm -p 8124:8123 yandex/clickhouse-server 
sleep 10
docker stop db_test1
docker stop db_test2

c976a7e50840653c112b1107fa0d95abb5af4635737ff03ec408f7adb24b86ac
30038fc56fbd0966363de0b886d4896cfb90f921a8385b7f30bc97c16ad232a4
db_test1
db_test2
